# Fizz Buzz

### Resources

This notebook is inspired/created by/from this [blog post](http://joelgrus.com/2016/05/23/fizz-buzz-in-tensorflow/) by Joel Grus.

### Note:

This isn't exactly a *serious* solution to a problem, but it's interesting+fun

In [1]:
# NOTE: this is a custom cell that contains the common imports I personally 
# use these may/may not be necessary for the following examples

# DL framework
import tensorflow as tf

from datetime import datetime

# common packages
import numpy as np
import os # handling file i/o
import sys
import math
import time # timing epochs
import random

# for ordered dict when building layer components
import collections

# plotting pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib import colors # making colors consistent
from mpl_toolkits.axes_grid1 import make_axes_locatable # colorbar helper


# from imageio import imread # read image from disk
# + data augmentation
from scipy import ndimage
from scipy import misc


import pickle # manually saving best params
from sklearn.utils import shuffle # shuffling data batches
from tqdm import tqdm # display training progress bar

# const
SEED = 42

# Helper to make the output consistent
def reset_graph(seed=SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# helper to create dirs if they don't already exist
def maybe_create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print("{} created".format(dir_path))
    else:
        print("{} already exists".format(dir_path))
    
def make_standard_dirs(saver=True, best_params=True, tf_logs=True):
    # `saver/` will hold tf saver files
    maybe_create_dir("saver")
    # `best_params/` will hold a serialized version of the best params
    # I like to keep this as a backup in case I run into issues with
    # the saver files
    maybe_create_dir("best_params")
    # `tf_logs/` will hold the logs that will be visable in tensorboard
    maybe_create_dir("tf_logs")

    
# set tf log level to supress messages, unless an error
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Important Version information
print("Python: {}".format(sys.version_info[:]))
print('TensorFlow: {}'.format(tf.__version__))

# Check if using GPU
if not tf.test.gpu_device_name():
    print('No GPU')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
reset_graph()

Python: (3, 5, 4, 'final', 0)
TensorFlow: 1.6.0-dev20180105
No GPU


In [2]:
make_standard_dirs()

saver created
best_params created
tf_logs created


In [ ]:
### Clean all logs
## WARNING! You likely don't want to do this (but if you do, this is a convenient call)
# !rm -r -f ./tf_logs/*

In [3]:
# these two functions (get_model_params and restore_model_params) are 
# ad[a|o]pted from; 
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb
def get_model_params():
    global_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {global_vars.op.name: value for global_vars, value in 
            zip(global_vars, tf.get_default_session().run(global_vars))}

def restore_model_params(model_params, g, sess):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: g.get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    sess.run(assign_ops, feed_dict=feed_dict)

# these two functions are used to manually save the best
# model params to disk
def save_obj(obj, name):
    with open('best_params/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('best_params/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

## Create Data

In [26]:
def fizz_buzz_encode(i):
    # one-hot representation of representations
    # fizz, buzz, fizzbuzz, and NA
    if i % 15 == 0:
        return np.array([0,0,0,1]) # fizzbuzz
    elif i % 5 == 0:
        return np.array([0,1,0,0]) # fizz
    elif i % 3 == 0:
        return np.array([0,0,1,0]) # buzz
    else:
        return np.array([1,0,0,0]) # none/NA

In [27]:
def fizz_buzz_decode(enc):
    if enc[1] == 1:
        return "fizz"
    elif enc[2] == 1:
        return "buzz"
    elif enc[3] == 1:
        return "fizzbuzz"

In [28]:
NUM_DIGITS = 10
tr_X = np.arange(101,2**NUM_DIGITS)
tr_y = np.array([fizz_buzz_encode(i) for i in tr_X])

In [32]:
print("{}; {}->{}".format(tr_X[1], tr_y[1], fizz_buzz_decode(tr_y[1])))

102; [0 0 1 0]->buzz


## Let's convert to binary

In [54]:
def binary_encode(i, NUM_DIGITS):
    bs = ("{0:0" + str(NUM_DIGITS) + "b}").format(i) # convert to bin rep
    bsl = [int(i) for i in bs] # convert bin rep to list of int
    return np.array(bsl)
#print("{}->{}".format(10, binary_encode(10, 10)))

10->[0 0 0 0 0 0 1 0 1 0]


In [55]:
tr_X = np.array([binary_encode(i, NUM_DIGITS) for i in tr_X])

we now have tr_X and tr_y where;

```tr_X = binary representation of int value
tr_y = encoded label; ie [0 0 1 0] = "buzz
```

## Build graph

In [ ]:
def build_graph():
    X = tf.placeholder()
    y = tf.placeholder()
    
    